Ejercicio del set de imágenes CIFAR10- Clasificando imágenes diversas
- PYTORCH
- Andrés de la Rosa

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

# New Section

In [0]:
#Importacion de todos los modulos
import time
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
import tensorflow as tf
import multiprocessing as mp
from keras.datasets import cifar10
import os

Esta vez cargo el CIFAR10 directamente de los datasets de Keras y no de la pagina de donde estaban originalmente como hice en el ejercicio de TensorFlow. Esto facilitó mucho el trabajo 
debido a que no tuve que definir las funciones que hacian el preprocesamiento de las imágenes. teniendo en cuenta que al momento de utilizar mis propias imágenes tendré que hacer dicho pre procesamiento

In [0]:
batch_size = 32
num_classes = 10
epochs = 5

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
# x_train - training data(images), y_train - labels(digits)
print(x_train.shape[0], 'imagenes de entrenamiento')
print(x_test.shape[0], 'imagenes de test')

50000 imagenes de entrenamiento
10000 imagenes de test


In [0]:
#Convirtiendo a one hot encoding
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Normalizando la entrada
x_train  /= 255
x_test /= 255

In [0]:
#Definiendo el modelo de 3 capas con relu y max pooling, haciendo el same padding para no perder pixeles
model = Sequential()

#Primera capa con relu y maxpooling
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Segunda capa con relu y maxpooling
model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Tercera capa con relu y maxpooling
model.add(Conv2D(128, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Poniendo los datos en formato flat
model.add(Flatten())

#Finalizando con los Fully connected layers
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [0]:
# Compilamos el modelo para calcular su acierto
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
opt = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=False)

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 14s 285us/step - loss: 1.6753 - acc: 0.3825 - val_loss: 1.3246 - val_acc: 0.5047
Epoch 2/5
50000/50000 [==============================] - 10s 209us/step - loss: 1.3117 - acc: 0.5250 - val_loss: 1.1724 - val_acc: 0.5729
Epoch 3/5
50000/50000 [==============================] - 10s 208us/step - loss: 1.1754 - acc: 0.5814 - val_loss: 1.0681 - val_acc: 0.6201
Epoch 4/5
50000/50000 [==============================] - 10s 207us/step - loss: 1.0960 - acc: 0.6114 - val_loss: 0.9140 - val_acc: 0.6817
Epoch 5/5
50000/50000 [==============================] - 10s 207us/step - loss: 1.0364 - acc: 0.6355 - val_loss: 0.9581 - val_acc: 0.6589


### Si sigo entrenando el modelo en mas epocas obtendré un mejor resultado, pero por cuestiones de ahorro de recursos solo lo entrené para 5 epocas 